<a href="https://colab.research.google.com/github/Steve-Falkovsky/Hypencoder-Entity-Linking/blob/Professional-Structure/notebooks/fine_tune_Hypencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import importlib.util

# for syncing changes to notebook when changing something in VScode
# deprecated/need to update to new IPython which breaks colab
# %load_ext autoreload
# %autoreload 2



REPO_NAME = "Hypencoder-Entity-Linking"
GIT_URL = f"https://github.com/Steve-Falkovsky/{REPO_NAME}.git"


# --- COLAB SETUP ---
is_colab = importlib.util.find_spec("google.colab") is not None
if is_colab:
    print("☁️ Running in Colab...")
    if not os.path.exists(REPO_NAME):
        !git clone {GIT_URL}

    # Move into the downloaded repo (The Root)
    os.chdir(REPO_NAME)



# --- LOCAL SETUP ---
else:
    print("💻 Running Locally...")
    if os.path.basename(os.getcwd()) == "notebooks":
        os.chdir("..")


%pip install -q -e "./hypencoder-paper"
os.chdir("./hypencoder-paper")

print(f"📍 Working Directory is now: {os.getcwd()}")
print("✅ Environment Ready!")

☁️ Running in Colab...
Cloning into 'Hypencoder-Entity-Linking'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (194/194), done.
Receiving objects: 100% (266/266), 705.72 KiB | 2.88 MiB/s, done.
remote: Total 266 (delta 116), reused 203 (delta 65), pack-reused 0 (from 0)
Resolving deltas: 100% (116/116), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 5.6 MB/s eta 0:00:00
📍 Working Directory is now: /content/Hypencoder-Entity-Linking/hypencoder-paper
✅ Environment Ready!


In [2]:
# loading the data
from datasets import load_dataset

dataset = load_dataset("Stevenf232/hypencoder_contrastiveLoss_nameOnly")
train_data = dataset['train']
val_data = dataset['validation']

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

(…)c5cdr_train_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

bc5cdr_val_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

bc5cdr_test_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2654 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2559 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2656 [00:00<?, ? examples/s]

In [3]:
# saving the data to a file
train_data.to_json('data/train.jsonl', lines=True)
val_data.to_json('data/val.jsonl', lines=True)

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

307335

In [4]:
# tokenizing the data before training

# training
!python hypencoder_cb/utils/tokenizer_utils.py \
--standard_format_jsonl='data/train.jsonl' \
--output_file='data/train_tokenized.jsonl' \
--tokenizer="cambridgeltl/SapBERT-from-PubMedBERT-fulltext" \
--add_special_tokens=True \
--query_max_length=32 \
--item_max_length=512

# validation
!python hypencoder_cb/utils/tokenizer_utils.py \
--standard_format_jsonl='data/val.jsonl' \
--output_file='data/val_tokenized.jsonl' \
--tokenizer="cambridgeltl/SapBERT-from-PubMedBERT-fulltext" \
--add_special_tokens=True \
--query_max_length=32 \
--item_max_length=512

tokenizer_config.json: 100% 198/198 [00:00<00:00, 1.67MB/s]
config.json: 100% 462/462 [00:00<00:00, 4.61MB/s]
vocab.txt: 226kB [00:00, 18.7MB/s]
special_tokens_map.json: 100% 112/112 [00:00<00:00, 1.06MB/s]
2654it [00:00, 382926.83it/s]
2it [00:00, 42.33it/s]
2it [00:00, 63.15it/s]
2654it [00:00, 54039.66it/s]
2559it [00:00, 391938.07it/s]
2it [00:00, 83.14it/s]
2it [00:00, 84.90it/s]
2559it [00:00, 46543.16it/s]


Everything in the output above is [00:00] which seems quite suspicious! (or it could just be really fast)



---



In [7]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext", use_fast=True)
print("cls", tok.cls_token, tok.cls_token_id)
print("sep", tok.sep_token, tok.sep_token_id)
print("pad", tok.pad_token, tok.pad_token_id)
print("bos", tok.bos_token, tok.bos_token_id)
print("eos", tok.eos_token, tok.eos_token_id)

cls [CLS] 2
sep [SEP] 3
pad [PAD] 0
bos None None
eos None None


Training the hypencoder

In [8]:
!python hypencoder_cb/train/train.py hypencoder_cb/train/configs/hypencoder.2_layer_SapBERT.yaml

2025-12-31 21:33:38.639316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767216818.659578   10343 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767216818.665965   10343 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767216818.682648   10343 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767216818.682672   10343 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767216818.682676   10343 computation_placer.cc:177] computation placer alr

In [11]:
%pip install huggingface_hub

# push the model to HuggingFace
# the model was saved in the model directory

from huggingface_hub import upload_folder

# Path where your model files (weights, config, etc.) are saved in Colab
# CHANGE THE CHECKPOINT NUMBER APPROPRIATELY
local_folder_path = "model/hypencoder.2_layer_SapBERT/checkpoint-400"

# Your desired repository ID on Hugging Face (e.g., "your-username/my-generic-model")
repo_id = "Stevenf232/SapBERT_freeze_hypencoder"

# You may need to create the repository first if it doesn't exist
from huggingface_hub import create_repo
create_repo(repo_id, exist_ok=True)

upload_folder(
    folder_path=local_folder_path,
    repo_id=repo_id,
    repo_type="model", # or "dataset" or "space"
    commit_message="Upload trained model from Colab"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...eckpoint-400/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

  ...ckpoint-400/rng_state.pth:  77%|#######7  | 11.3kB / 14.6kB            

  ...int-400/pytorch_model.bin:   0%|          | 1.53MB /  483MB            

  ...eckpoint-400/optimizer.pt:   1%|          |  579kB / 89.8MB            

  ...int-400/training_args.bin:   6%|6         |   368B / 5.91kB            

CommitInfo(commit_url='https://huggingface.co/Stevenf232/SapBERT_freeze_hypencoder/commit/8f8fc2b8a653287206056c00872c6e6eafbad5bc', commit_message='Upload trained model from Colab', commit_description='', oid='8f8fc2b8a653287206056c00872c6e6eafbad5bc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Stevenf232/SapBERT_freeze_hypencoder', endpoint='https://huggingface.co', repo_type='model', repo_id='Stevenf232/SapBERT_freeze_hypencoder'), pr_revision=None, pr_num=None)